In [1]:
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import env
import os

In [2]:
# acquire the data from local drive
#  create a function for it
def acquire(file_name, column_names):
    cohort=get_cohort()
    df=pd.read_csv(file_name, sep="\s", 
                       header=None, 
                       names=column_names, 
                       usecols=[0, 2, 3,4, 5]) 
    cohort=cohort.rename(columns={'id':'cohort_id'})
    df=pd.merge(cohort,df,on='cohort_id')
    df=df.drop(columns=['deleted_at'])
    return df

In [3]:
def get_connection(db, user=env.user, host=env.host, password=env.password):
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'

def get_cohort():
    filename = "cohort.csv"
    if os.path.isfile(filename):
        return pd.read_csv(filename)
    else:
        # read the SQL query into a dataframe
        df = pd.read_sql('SELECT * FROM cohorts', get_connection('curriculum_logs'))

        # Write that dataframe to disk for later. Called "caching" the data for later.
        df.to_csv(filename,index=False)

        # Return the dataframe to the calling code
        return df  

In [4]:
# clean up the columns and set date to datetime
# change date to index
def prep(df, user):
    df = df[df.user_id == user]
    df.date = pd.to_datetime(df.date)
    df = df.set_index(df.date)
    return df

In [5]:
file_name = "anonymized-curriculum-access.txt"
column_names = ['date', 'endpoint', 'user_id', 'cohort_id', 'source_ip']
df = acquire(file_name, column_names)

In [6]:
# check missing value
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 847330 entries, 0 to 847329
Data columns (total 12 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   cohort_id   847330 non-null  int64 
 1   name        847330 non-null  object
 2   slack       847330 non-null  object
 3   start_date  847330 non-null  object
 4   end_date    847330 non-null  object
 5   created_at  847330 non-null  object
 6   updated_at  847330 non-null  object
 7   program_id  847330 non-null  int64 
 8   date        847330 non-null  object
 9   endpoint    847329 non-null  object
 10  user_id     847330 non-null  int64 
 11  source_ip   847330 non-null  object
dtypes: int64(3), object(9)
memory usage: 84.0+ MB


In [7]:
# the cohort_id alone has 900223 value count
df.cohort_id

0           1
1           1
2           1
3           1
4           1
         ... 
847325    139
847326    139
847327    139
847328    139
847329    139
Name: cohort_id, Length: 847330, dtype: int64

In [8]:
# it looks like users without cohort_id are all the same, which means some users didn't assigned a cohort_id
df[df.user_id==48].cohort_id

Series([], Name: cohort_id, dtype: int64)

In [9]:
# check if they have spread out date
df[df.cohort_id.isnull()].date.value_counts()

Series([], Name: date, dtype: int64)

In [10]:
# now get cohort from sql
cohort=get_cohort()

In [11]:
# retireve data
cohort=cohort.rename(columns={'id':'cohort_id'})

In [12]:
# cohort ahas 53 unique
cohort.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53 entries, 0 to 52
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   cohort_id   53 non-null     int64  
 1   name        53 non-null     object 
 2   slack       53 non-null     object 
 3   start_date  53 non-null     object 
 4   end_date    53 non-null     object 
 5   created_at  53 non-null     object 
 6   updated_at  53 non-null     object 
 7   deleted_at  0 non-null      float64
 8   program_id  53 non-null     int64  
dtypes: float64(1), int64(2), object(6)
memory usage: 3.9+ KB


In [13]:
# while the data frame has 47
df.user_id.nunique()

911

In [14]:
# change id name to cohort id
df=pd.merge(df,cohort,on='cohort_id')

In [15]:
# zero missing values, drop deleted_at
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 847330 entries, 0 to 847329
Data columns (total 20 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   cohort_id     847330 non-null  int64  
 1   name_x        847330 non-null  object 
 2   slack_x       847330 non-null  object 
 3   start_date_x  847330 non-null  object 
 4   end_date_x    847330 non-null  object 
 5   created_at_x  847330 non-null  object 
 6   updated_at_x  847330 non-null  object 
 7   program_id_x  847330 non-null  int64  
 8   date          847330 non-null  object 
 9   endpoint      847329 non-null  object 
 10  user_id       847330 non-null  int64  
 11  source_ip     847330 non-null  object 
 12  name_y        847330 non-null  object 
 13  slack_y       847330 non-null  object 
 14  start_date_y  847330 non-null  object 
 15  end_date_y    847330 non-null  object 
 16  created_at_y  847330 non-null  object 
 17  updated_at_y  847330 non-null  object 
 18  dele